## Top 10 Airline scrape tables

scraper to get top 10 airline information on different airline stats
https://en.wikipedia.org/wiki/Largest_airlines_in_the_world

In [1]:
# Import necessary packages
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re
import lxml
import bs4.builder._lxml

# Site URL
url = 'https://en.wikipedia.org/wiki/Largest_airlines_in_the_world'

# Make a GET request to fetch the raw HTML content
html_content = requests.get(url).text

# Parse HTML code for the entire site
soup = BeautifulSoup(html_content, "html.parser")


#print(soup.prettify()) # print the parsed data of html

In [2]:
gdp = soup.find_all("table", attrs={"class": "wikitable sortable"})
print("Number of tables on site: ",len(gdp))

Number of tables on site:  10


In [25]:
#table1 - revenue
revenue = gdp[0]
body = revenue.find_all("tr")
head = body[0]
body_rows = body[1:]

body_rows

[<tr>
 <th colspan="4">(US$ billions)
 </th></tr>,
 <tr>
 <th>1
 </th>
 <td align="left"><a href="/wiki/Delta_Air_Lines" title="Delta Air Lines">Delta Air Lines</a></td>
 <td style="text-align:left;"><span class="flagicon"><img alt="" class="thumbborder" data-file-height="650" data-file-width="1235" decoding="async" height="12" src="//upload.wikimedia.org/wikipedia/en/thumb/a/a4/Flag_of_the_United_States.svg/23px-Flag_of_the_United_States.svg.png" srcset="//upload.wikimedia.org/wikipedia/en/thumb/a/a4/Flag_of_the_United_States.svg/35px-Flag_of_the_United_States.svg.png 1.5x, //upload.wikimedia.org/wikipedia/en/thumb/a/a4/Flag_of_the_United_States.svg/46px-Flag_of_the_United_States.svg.png 2x" width="23"/> </span><a href="/wiki/United_States" title="United States">United States</a>
 </td>
 <td>44.9</td>
 <td>4.1</td>
 <td>61.8</td>
 <td>38.1</td>
 <td>89,000
 </td></tr>,
 <tr>
 <th>2
 </th>
 <td align="left"><a href="/wiki/American_Airlines_Group" title="American Airlines Group">America

In [19]:
headings = []
for item in head.find_all("th"): # loop through all th elements
    # convert the th elements to text and strip "\n"
    item = (item.text).strip("\n|\n|\t")
    # append the clean column name to headings
    headings.append(item)

#As we only need the airline name we will drop anything that doesn't have it to avoid annoying wrangling issues

headings = ['Airline']

print(headings)


['Airline']


In [20]:
# Next is now to loop though the rest of the rows
#print(body_rows[0])
all_rows = [] # will be a list for list for all rows
for row_num in range(len(body_rows)): # A row at a time
    row = [] # this will old entries for one row
    for row_item in body_rows[row_num].find_all("td"): #loop through all row entries
        # regex to remove \xa0 and \n and comma from row_item.text
        text = re.sub("(\n)|,|\n|\t|IATA  – ","",row_item.text)# \n is the newline \t is a weird wikipedia thing and comma
        #append aa to row - note one row entry is being appended
        row.append(text)
    # append one row to all_rows
    all_rows.append(row)

In [21]:
# We can now use the data on all_rowsa and headings to make a table
# all_rows becomes our data and headings the column names
df = pd.DataFrame(data=all_rows,columns=headings)
df.head()

ValueError: 1 columns passed, passed data had 0 columns

In [9]:
df.replace({'ICAO' : { '' : np.nan, 'n/a' : np.nan}}, inplace = True)

df = df[df.ICAO.notnull()]

df['ICAO'].isnull().any()

False

In [10]:
df

,IATA,ICAO,Airline,Call sign,Country/Region,Comments
1,,GNL,135 Airways,GENERAL,United States,
14,2T,TBS,Timbis Air,,Kenya,
15,,WYT,2 Sqn No 1 Elementary Flying Training School,WYTON,United Kingdom,Royal Air Force
16,,TFU,213th Flight Unit,THJY,Russia,State Airline
17,,CHD,223rd Flight Unit,CHKALOVSK-AVIA,Russia,State Airline
...,...,...,...,...,...,...
6112,,CYN,Zhongyuan Aviation,ZHONGYUAN,China,ICAO Code and callsign no longer allocated
6113,3J,WZP,Zip,ZIPPER,Canada,ICAO Code and callsign no longer allocated
6114,ZG,TZT,ZIPAIR Tokyo,ZIPPY,Japan,Subsidiary of Japan Airlines
6115,Z4,OOM,Zoom Airlines,ZOOM,Canada,defunct ICAO Code and callsign no longer alloc...


### can I scrape cancelled flight for the last 30 days?